[View in Colaboratory](https://colab.research.google.com/github/guilgautier/DPPy/blob/docs/notebooks/Tuto_DPPy.ipynb)

# Why are we here ?
---

## DPPy
  + Acronym: DPP + Python
  + Python Toolbox for (sampling) DPPs
 
## Goal
  + showcase the DPP samplers featured in DPPy
  + present tools behind the scene 
  
  [GitHub](https://github.com/guilgautier/DPPy)
  [![Documentation Status](https://readthedocs.org/projects/dppy/badge/?version=latest)](https://dppy.readthedocs.io/en/latest/?badge=latest)
  [![Build Status](https://travis-ci.com/guilgautier/DPPy.svg?branch=master)](https://travis-ci.com/guilgautier/DPPy)
  + needs/suggestion for improvement

# Program of the day
---

## I.   Check installation
## II.   Sample DPPs with DPPy
## III.   Tools behind the scene
## IV.   Suggestions/needs

# I. Check installation
---

[Instructions](https://github.com/guilgautier/DPPy#installation)

### Any issue?

- [Travis](https://travis-ci.com/guilgautier/DPPy) says **no** :-)
    [![Build Status](https://travis-ci.com/guilgautier/DPPy.svg?branch=master)](https://travis-ci.com/guilgautier/DPPy)




In [ ]:
#@title ## Who is *using*:

GitHub = 1 #@param {type:"slider", min:0, max:20, step:1}
git = 1 #@param {type:"slider", min:0, max:20, step:1}
none = 0 #@param {type:"slider", min:0, max:20, step:1}


### Ready?

```bash
(cd DPPy)
jupyter notebook
```

In [ ]:
!rm -r DPPy
!git clone https://github.com/guilgautier/DPPy.git

In [ ]:
!pip install scipy --upgrade
!pip install DPPy/.

# II. Sample DPPs with DPPy
----

## 1. Continuous DPPs: $\beta$-Ensembles
  + Hermite, Laguerre, Jacobi, Circular, Ginibre
  + matrix models (full/tridiagonal)

## 2. Finite DPPs

  + exact sampling
  + approximate sampling

## 3. Exotic DPPs
  + Uniform Spanning Trees
  + Plancherel (RSK, ...)
  + Descent Processes

## Main DPPy objects' methods

#### `.sample()`
#### `.plot()`

## II. 1. Continuous DPPs: $\beta$-Ensembles

### See [documentation](https://dppy.readthedocs.io/en/latest/continuous_dpps/index.html)

$$
(x_1,\dots,x_N) 
	\sim 
		\frac{1}{Z_{N,\beta}}
		\left|\Delta(x_1,\dots,x_N)\right|^{\beta}
		\prod_{i= 1}^N 
			\mu(d x_i)
$$

- Random matrix models
- $\beta=2$: projection DPPs

In [ ]:
from dppy.beta_ensembles import *

$$
(x_1,\dots,x_N) 
	\sim 
		\frac{1}{Z_{N,\beta}}
		\left|\Delta(x_1,\dots,x_N)\right|^{\beta}
		\prod_{i= 1}^N 
			\mu(d x_i)
$$

- Random matrix models
- $\beta=2$: projection DPPs


### d. Circular

In [ ]:
circular = BetaEnsemble("circular", beta=2)

#### Visualization of the cristallization as $\beta$ increases

In [ ]:
#@title ##### Use a slider!

_beta = 25 #@param {type:"slider", min:0, max:100, step:1}
_size = 63 #@param {type:"slider", min:0, max:100, step:1}

circular.beta = _beta
banded_params = {"size":_size}
circular.sample("banded", **banded_params)
circular.plot()

##### Or simply a loop

In [ ]:
banded_params = {"size":30}
for _beta in (1, 10, 50, 100):
  circular.beta = _beta
  circular.sample("banded", **banded_params)
  circular.plot()

#### Sample using full matrix model

In [ ]:
circular.beta = 1
full_params = {"N":30, "haar_mode":"QR"}
circular.sample("full", **full_params)
circular.plot()

#### Sample using banded (tridiagonal) matrix model

In [ ]:
circular.beta = 10
banded_params = {"size":300}
circular.sample("banded", **banded_params)
circular.hist()

### a. Hermite $\mu = \mathcal{N}$

In [ ]:
hermite = BetaEnsemble("hermite", beta=2)

#### Sample using full matrix model

In [ ]:
full_params = {"N":1000}
hermite.sample("full", **full_params)
hermite.hist()

#### Sample using banded (tridiagonal) matrix model

In [ ]:
banded_params = {"loc":0.0, "scale":np.sqrt(2), "size":1000}
hermite.sample("banded", **banded_params)
hermite.hist()


### b. Laguerre $\mu = \mathcal{\Gamma}$

In [ ]:
laguerre = BetaEnsemble("laguerre", beta=2)

#### Sample using full matrix model

In [ ]:
full_params = {"M":1500, "N":1000}
laguerre.sample("full", **full_params)
laguerre.hist()

#### Sample using banded (tridiagonal) matrix model

In [ ]:
banded_params = {"shape":10000, "scale":2.0, "size":2000}
laguerre.sample("banded", **banded_params)
laguerre.hist()

### c. Jacobi $\mu = \operatorname{Beta}$

In [ ]:
jacobi = BetaEnsemble("jacobi", beta=2)

#### Sample using full matrix model

In [ ]:
full_params = {"M_1":1500, "M_2":1200, "N":1000}
jacobi.sample("full", **full_params)
jacobi.hist()

#### Sample using banded (tridiagonal) matrix model

In [ ]:
banded_params = {"a":100, "b":100, "size":1000} 
jacobi.sample("banded", **banded_params)
jacobi.hist()

## II. 2. Discrete DPPs

### See [documentation](https://dppy.readthedocs.io/en/latest/finite_dpps/index.html)

In [ ]:
from dppy.finite_dpps import *

### Build inclusion kernel $\mathbf{K}$

$\operatorname{DPP}(\mathbf{K})$, with $\mathbf{K}\in\mathbb{R}^{N\times N}$ 

$$
	\mathbf{K} = \sum_{n=1}^{N} \lambda_n u_n u_n^{\top}
$$

In [ ]:
r, N = 10, 10 #

# Random orthogonal vectors
A = np.random.randn(r, N)
eig_vecs, _ = la.qr(A.T, mode="economic")
# Random eigenvalues
eig_vals = np.random.rand(r) # 0< <1
#eig_vals = np.random.choice([0.0, 1.0], size=r)# 0 or 1 i.e. projection

K = eig_vecs*eig_vals @ eig_vecs.T

### Declare a finite DPP in DPPy

#### 1. Via eigen-decomposition

In [ ]:
DPP = FiniteDPP("inclusion", **{"K_eig_dec":(eig_vals, eig_vecs)})
#print(DPP.K)

#### 2. Via its kernel

In [ ]:
DPP = FiniteDPP("inclusion", **{"K":K})

### Exact sampling scheme

- $\operatorname{DPP}(\mathbf{K})$, with $\mathbf{K}\in\mathbb{R}^{N\times N}$ 

$$
	\mathbf{K} = \sum_{n=1}^{N} \lambda_n u_n u_n^{\top}
$$

1. Draw independent $\operatorname{\mathcal{B}er}(\lambda_n)$ for each eigenvector $u_n$ and store the selected ones in $\tilde{U}$.
2. Sample from the corresponding *projection* $\operatorname{DPP}(\tilde{U}\tilde{U}^{\top})$.

In [ ]:
# Sample
for _ in range(10):
  DPP.sample_exact()

DPP.list_of_samples

In [ ]:
DPP.flush_samples()
DPP.list_of_samples

### Other features


#### a. Compute the *other* kernel 
- $L=K(I-K)^{-1}$

In [ ]:
DPP = FiniteDPP("inclusion", **{"K_eig_dec":(eig_vals, eig_vecs)})
print(DPP.L)
DPP.compute_L()

- $K=L(I+L)^{-1}$

In [ ]:
eig_vals = 4*np.random.rand(r) # >=0
DPP = FiniteDPP("marginal", **{"L_eig_dec":(eig_vals, eig_vecs)})
print(DPP.L)
DPP.compute_K()

#### b. Compute/plot the underlying kernel


In [ ]:
eig_vals = np.random.rand(r) # 0< <1
DPP = FiniteDPP("inclusion", **{"K_eig_dec":(eig_vals, eig_vecs)})
DPP.plot_kernel()

## II. 3. Exotic DPPs
### See [documentation](https://dppy.readthedocs.io/en/latest/exotic_dpps/index.html)

- Uniform spanning trees
- Plancherel
- Descent Processes

In [ ]:
from dppy.exotic_dpps import *

### a. Uniform Spanning Trees

In [ ]:
g = nx.Graph()
edges = [(0,2), (0,3), (1,2), (1,4), (2,3), (2,4), (3,4)]
g.add_edges_from(edges)

ust = UST(g)

ust.plot_graph()

#### Display kernel

In [ ]:
ust.compute_kernel()
ust.plot_kernel()

#### Sample a UST

In [ ]:
for md in ('Aldous-Broder', 'Wilson', 'DPP_exact'):
    ust.sample(md); ust.plot()

# III. Tools behind the scene

---


 - Host collaborative project [GitHub](https://github.com/guilgautier/DPPy)
 - Documentation [![Documentation Status](https://readthedocs.org/projects/dppy/badge/?version=latest)](https://dppy.readthedocs.io/en/latest/?badge=latest)
 - Continuous integration [![Build Status](https://travis-ci.com/guilgautier/DPPy.svg?branch=master)](https://travis-ci.com/guilgautier/DPPy)
 
 
 #### Reproducible reasearch
 - [DPPy](https://github.com/guilgautier/DPPy)
 - companion paper [DPPy_paper](https://github.com/guilgautier/DPPy_paper)

# IV.   Suggestions/needs?

---


*   
*
*
*
*
*   
*
*
*
*